# LAB4 - КРИПТОГРАФИЈА

## 151027 Fisnik Limani

# 1. DES (DATA ENCRYPTION STANDARD) IMPLEMENTATION

In [130]:
# Convert ASCII text to binary
def _ascii_to_bin(text):

    out = ""
    for letter in text:
        out += bin(ord(letter))[2:].zfill(8)
    # ord(letter) - Given a string of length one, return an integer representing the Unicode code point of the character 
    #               when the argument is a unicode object, or the value of the byte when the argument is an 8-bit string. 

    # Pad block with 0s if necessary
    for i in range(64-(len(out))):
        out += "0"

    return out

# Convert binary to ASCII text
def _bin_to_ascii(bit64):

    out = ""
    for i in range(0, len(bit64), 8):
        out += chr(int(bit64[i:i+8], 2))

    #out.replace("\x00", "")
    return out

# Convert HEX text to binary
def _hex_to_bin(text):

    out = ""
    for letter in text:
        out += bin(int(letter, 16))[2:].zfill(4)

    # Pad block with 0s if necessary
    for i in range(64-(len(out))):
        out += "0"

    return out

# Convert binary to hex
def _bin_to_hex(bit64):

    out = ""
    for i in range(0, len(bit64), 4):
        out += hex(int(bit64[i:i+4], 2))[2:]

    return out

"""
Expansion:

    - Takes 32-bit binary number in the form of a string
    - Transforms and permutes 32-bit number into 48-bit number
    - Returns 48-bit binary number in the form of a string

"""
def _expansion(bit32):

    # Expansion table
    e = [32,  1,  2,  3,  4,  5,
          4,  5,  6,  7,  8,  9,
          8,  9, 10, 11, 12, 13,
         12, 13, 14, 15, 16, 17,
         16, 17, 18, 19, 20, 21,
         20, 21, 22, 23, 24, 25,
         24, 25, 26, 27, 28, 29,
         28, 29, 30, 31, 32,  1]

    # Perform permutation on bit32
    out = ""
    for number in e:
        out += bit32[number-1]

    return out

"""
Sbox Substitution:

    - Takes 48-bit binary number in the form of a string
    
    - Splits 48-bit input into 8 chunks of 6 bits
    - Performs substitutions on each chunk using 8 hard-coded tables:
	    > The first and last bit of the 6-bit chunk acts as a
	      coordinate for the row of the Sbox
	    > The middle 4 bits act as a coordinate for the column
	    
    - Returns 32-bit binary number in the form of a string 
"""
def _Sbox(bit48, s_boxovi = False, round_s_boxovi = False, round_s_boxovi_list = None):

    s = [
         # s1
         [[14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
          [ 0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8],
          [ 4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0],
          [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]],
         # s2
         [[15,  1,  8, 14,  6, 11,  3,  4,  9,  7,  2, 13, 12,  0,  5, 10],
          [ 3, 13,  4,  7, 15,  2,  8, 14, 12,  0,  1, 10,  6,  9, 11,  5],
          [ 0, 14,  7, 11, 10,  4, 13,  1,  5,  8, 12,  6,  9,  3,  2, 15],
          [13,  8, 10,  1,  3, 15,  4,  2, 11,  6,  7, 12,  0,  5, 14,  9]],
         # s3
         [[10,  0,  9, 14,  6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8],
          [13,  7,  0,  9,  3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1],
          [13,  6,  4,  9,  8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7],
          [ 1, 10, 13,  0,  6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12]],
         # s4
         [[ 7, 13, 14,  3,  0,  6,  9, 10,  1,  2,  8,  5, 11, 12,  4, 15],
          [13,  8, 11,  5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,  9],
          [10,  6,  9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4],
          [ 3, 15,  0,  6, 10,  1, 13,  8,  9,  4,  5, 11, 12,  7,  2, 14]],
         # s5
         [[ 2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9],
          [14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3,  9,  8,  6],
          [ 4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14],
          [11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3]],
         # s6
         [[12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11],
          [10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8],
          [ 9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6],
          [ 4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13]],
         # s7
         [[ 4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1],
          [13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6],
          [ 1,  4, 11, 13, 12,  3,  7, 14, 10, 15,  6,  8,  0,  5,  9,  2],
          [ 6, 11, 13,  8,  1,  4, 10,  7,  9,  5,  0, 15, 14,  2,  3, 12]],
         # s8
         [[13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7],
          [ 1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2],
          [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8],
          [ 2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11]]
        ]
    
    # Break up 48-bit input into 8 6-bit chunks
    bit6_list = []
    for i in range(0, 48, 6):
        bit6_list.append(bit48[i:i+6])
    if s_boxovi:
        print(bit6_list)
    elif round_s_boxovi:
        # print(bit6_list)
        round_s_boxovi_list.append(bit6_list)

    # Perform 8 Sbox substitutions using the 8 6-bit chunks
    # and the 8 Sbox tables s1-s8
    out = ""
    for i in range(8):
        chunk = bit6_list[i]
        row = int(chunk[0]+chunk[5], 2)
        col = int(chunk[1:5], 2)

        out += (bin(s[i][row][col])[2:].zfill(4))

    return out

"""
Permute:

    - Takes 32-bit binary number in the form of a string
    - Performs simple permutation on the number
    - Returns 32-bit binary number in the form of a string

"""
def _permute(bit32):

    # Permutation table
    p = [16,  7, 20, 21,
         29, 12, 28, 17,
          1, 15, 23, 26,
          5, 18, 31, 10,
          2,  8, 24, 14,
         32, 27,  3,  9,
         19, 13, 30,  6,
         22, 11,  4, 25]

    # Perform permutation on bit32
    out = ""
    for number in p:
        out += bit32[number-1]

    return out

"""
f Function:

    - Takes 32-bit binary number in the form of a string (bit32)
      and a 48-bit binary number in the form of a string (key)

    - Performs various permutations, expansions, and reductions
      on the input bit32

    - Returns 32-bit binary number in the form of a string


"""
def f(bit32, key, s_boxovi = False, round_s_boxovi = False, round_s_boxovi_list = None):

    # Expand 32-bit input to 48 bits and convert to int
    bit48 = int(_expansion(bit32), base = 2)


    # XOR 48-bit result and round key, and then convert back to binary
    bit48 ^= int(key, 2)
    bit48 = bin(bit48)[2:].zfill(48)
    # [2:] - because bin gives a number of a form '0b{binaryNumberRepresentation}'
    # zfill(width) - The width specifies the length of the returned string from zfill() with '0' digits filled to the left.
    

    # Perform Sbox substitution
    bit32 = _Sbox(bit48, s_boxovi, round_s_boxovi, round_s_boxovi_list)


    # Perform permutation on bit32
    bit32 = _permute(bit32)
    

    return bit32

def _cipher(bit64, keys, first_round = False, s_boxovi = False, round_s_boxovi = False, round_s_boxovi_list = None):

    # Initial permutation table
    ip = [58, 50, 42, 34, 26, 18, 10, 2,
          60, 52, 44, 36, 28, 20, 12, 4,
          62, 54, 46, 38, 30, 22, 14, 6,
          64, 56, 48, 40, 32, 24, 16, 8,
          57, 49, 41, 33, 25, 17,  9, 1,
          59, 51, 43, 35, 27, 19, 11, 3,
          61, 53, 45, 37, 29, 21, 13, 5,
          63, 55, 47, 39, 31, 23, 15, 7]

    # Final permutation
    fp = [40,  8, 48, 16, 56, 24, 64, 32,
          39,  7, 47, 15, 55, 23, 63, 31,
          38,  6, 46, 14, 54, 22, 62, 30,
          37,  5, 45, 13, 53, 21, 61, 29,
          36,  4, 44, 12, 52, 20, 60, 28,
          35,  3, 43, 11, 51, 19, 59, 27,
          34,  2, 42, 10, 50, 18, 58, 26,
          33,  1, 41,  9, 49, 17, 57, 25]
    

    # Perform initial permutation on block
    p = ""
    for number in ip:
        p += bit64[number-1]

    # Split block into two halves
    ln = p[:32]
    rn = p[32:]
    
    # print('After initial permutation', _bin_to_hex(p))
    # print('L0', _bin_to_hex(ln), 'R0', _bin_to_hex(rn))
        
    # 16 rounds of encryption, rotating and XORing each half after each round
    for i in range(16):
        temp = rn
        if round_s_boxovi:
            rn = bin(int(ln, 2) ^ int(f(rn, keys[i], round_s_boxovi = round_s_boxovi, round_s_boxovi_list = round_s_boxovi_list), 2))[2:].zfill(32)
        elif i == 1 and s_boxovi:
            rn = bin(int(ln, 2) ^ int(f(rn, keys[i], True), 2))[2:].zfill(32)
            break
        elif i == 0 and first_round:
            print(ln + rn)
            break
        else:
            rn = bin(int(ln, 2) ^ int(f(rn, keys[i]), 2))[2:].zfill(32)
        ln = temp
        # print(_bin_to_hex(ln), _bin_to_hex(rn), _bin_to_hex(keys[i]))
    
    # We reverse the order of the two blocks into R16L16 and perform final permutation
    combined = rn + ln
    # combined = ln + rn
    out = ""
    for number in fp:
        out += combined[number-1]
    
    return out


import random

"""
DES Key Scheduler:

    - Generates 16 48-bit subkeys from a randomly
      generated 64-bit initial key

    - Returns a list of the 16 subkeys in the form
      of binary strings

"""

def key_scheduler(key = None, is_key_random = False):

    # -------------------- Variables -------------------- #
    #
    # k         Initial 64-bit key
    # k_prime   Permuted 56-bit key
    # c0        Left half of k_prime
    # d0        Right half of k_prime
    # c_keys    16 28-bit permutations of c0 using lrt
    # d_keys    16 28-bit permutations of d0 using lrt
    # keys      Final list of 16 48-bit keys
    #
    # --------------------------------------------------- #

    
    # ---------------- Hard-Coded Values ---------------- #

    # pc1       Permutation table No. 1
    pc1 = [57, 49, 41, 33, 25, 17,  9, #7
            1, 58, 50, 42, 34, 26, 18, #14
           10,  2, 59, 51, 43, 35, 27, #21
           19, 11,  3, 60, 52, 44, 36, #28
           63, 55, 47, 39, 31, 23, 15, #35
            7, 62, 54, 46, 38, 30, 22, #42
           14,  6, 61, 53, 45, 37, 29, #49
           21, 13,  5, 28, 20, 12,  4] #56

    # pc2       Permutation table No. 2
    pc2 = [14, 17, 11, 24,  1,  5,
           3,  28, 15,  6, 21, 10,
           23, 19, 12,  4, 26,  8,
           16,  7, 27, 20, 13,  2,
           41, 52, 31, 37, 47, 55,
           30, 40, 51, 45, 33, 48,
           44, 49, 39, 56, 34, 53,
           46, 42, 50, 36, 29, 32]
           

    # lrt       Left rotation table
    lrt = [1, 1, 2, 2, 2, 2, 2, 2,
           1, 2, 2, 2, 2, 2, 2, 1]

    # --------------------------------------------------- #


    

    # Generate random 64-bit number
    # Note - Even though the algorithm starts with a 64-bit key, DES is still technically
    # 56-bit, because 8 of the bits are used as parity bits. These bits are lost during the
    # first permutation using pc1.
    k = None
    if is_key_random:
        k = random.getrandbits(64)    # getrandbits(k) - Returns a python long int with k random bits. 
        k = bin(k)[2:].zfill(64)
    else:
        k = _hex_to_bin(key)
        # print('key: ',key)
        # print('k: ', k)
    

    # Generate k_prime using permutation table 1
    k_prime = ""
    for number in pc1:
        k_prime += k[number-1]
    
    # print("k_prime: ", k_prime)

    # Find c0 (left half of k_prime) and d0 (right half of k_prime)
    c0 = k_prime[:28]
    d0 = k_prime[28:]


    # Rotate c0 and d0 according to left rotation table
    c_keys = []
    d_keys = []
    for i in range(16):
        num = lrt[i]
        
        temp = c0[0:num]
        c0 = c0[num:] + temp
        c_keys.append(c0)

        temp = d0[0:num]
        d0 = d0[num:] + temp
        d_keys.append(d0)
        # print(i, c0, d0)


    # Generate subkeys by combining c_keys[i] and d_keys[i].
    # Use permutation table 2 to generate 16 48-bit keys
    keys = []
    for i in range(16):
        cat = c_keys[i] + d_keys[i]
        # print('cat:', cat)
        temp = ""
        for number in pc2:
            temp += cat[number-1]
        keys.append(temp)

    return keys

def _plaintext(bit64, keys):

    # Initial permutation table
    ip = [58, 50, 42, 34, 26, 18, 10, 2,
          60, 52, 44, 36, 28, 20, 12, 4,
          62, 54, 46, 38, 30, 22, 14, 6,
          64, 56, 48, 40, 32, 24, 16, 8,
          57, 49, 41, 33, 25, 17,  9, 1,
          59, 51, 43, 35, 27, 19, 11, 3,
          61, 53, 45, 37, 29, 21, 13, 5,
          63, 55, 47, 39, 31, 23, 15, 7]

    # Final permutation table
    fp = [40,  8, 48, 16, 56, 24, 64, 32,
          39,  7, 47, 15, 55, 23, 63, 31,
          38,  6, 46, 14, 54, 22, 62, 30,
          37,  5, 45, 13, 53, 21, 61, 29,
          36,  4, 44, 12, 52, 20, 60, 28,
          35,  3, 43, 11, 51, 19, 59, 27,
          34,  2, 42, 10, 50, 18, 58, 26,
          33,  1, 41,  9, 49, 17, 57, 25]


    # Perform initial permutation on block
    p = ""
    for number in ip:
        p+= bit64[number-1]


    # Split block into two halves
    ln = p[:32]
    rn = p[32:]

    # 16 rounds of decryption, rotating and XORing each half after each round
    # Notice that for decryption, the algorithm starts with the left half as
    # opposed to the right half with encryption, and it iterates backwards
    # through the key set.
    for i in range(15, -1, -1):
        temp = rn
        rn = bin(int(ln, 2) ^ int(f(rn, keys[i]), 2))[2:].zfill(32)
        ln = temp

    # Combine left and right halves and perform final permutation
    # combined = ln + rn
    combined = rn + ln
    out = ""
    for number in fp:
        out += combined[number-1]


    return out

In [232]:
def encrypt(text, keys, first_round=False, s_boxovi = False, round_s_boxovi = False, round_s_boxovi_list = None):
    text = _bin_to_hex(_ascii_to_bin(text))
    # print("ENCRYPT", text)
    # Split text up into 64-bit chunks
    blocks = []
    
    for i in range(0, len(text), 16):
        blocks.append(text[i:i+16])
    # print(blocks)

    # Encrypt chunks
    e_list = []
    for block in blocks:
        e_list.append(_cipher(_hex_to_bin(block), keys, first_round, s_boxovi, round_s_boxovi, round_s_boxovi_list))


    # Combine chunks back into a single string
    out = ""
    for block in e_list:
        # out += _bin_to_ascii(block)    # ASCII output
        out += hex(int(block, 2))[2:]      # Hex output
        #out += block                    # Binary output
    return out

In [233]:
def decrypt(cipher, keys):
    cipher = _hex_to_bin(cipher)
    # print(cipher)
    # Split text up into 64-bit chunks
    blocks = []
    for i in range(0, len(cipher), 64):
        blocks.append(cipher[i:i+64])


    # Decrypt chunks
    d_list = []
    for block in blocks:
        d_list.append(_plaintext(block, keys))


    # Combine chunks back into a single string
    out = ""
    for block in d_list:
        out += _bin_to_hex(block)


    return _bin_to_ascii(_hex_to_bin(out))

# 1.1. TESTS FOR DES IMPLEMENTATION

In [254]:
k = 'FFFFFFFFFFFFFFFF'
keys = key_scheduler(k)

message = 'F.Limani'
enkriptirana = encrypt(message, keys).upper()
print('Enkriptirana poraka na', message, 'e', enkriptirana)

dekriptirana = decrypt(enkriptirana, keys)
print('Dekriptirana poraka na', enkriptirana, dekriptirana)

Enkriptirana poraka na F.Limani e 1D806E75759165D1
Dekriptirana poraka na 1D806E75759165D1 F.Limani


In [255]:
k = '123456789ABCDEF'
keys = key_scheduler(k)

message = 'Crypto..'
enkriptirana = encrypt(message, keys).upper()
print('Enkriptirana poraka na', message, 'e', enkriptirana)

dekriptirana = decrypt(enkriptirana, keys)
print('Dekriptirana poraka na', enkriptirana, dekriptirana)

Enkriptirana poraka na Crypto.. e 4C73BED5BF8B4D1E
Dekriptirana poraka na 4C73BED5BF8B4D1E Crypto..


# 2. FUNCTION padd_message(message)

- Function that fills the message (input) with 0 if it is not divisible by 8

In [146]:
def padd_message(message):
    """
        Function that padds the message with 0s if the number of characters is not a divisor of 8
    """
    l = len(message)
    if l % 8 == 0:
        return message
    while l % 8 != 0:
        # print(message)
        message += '0'
        l = len(message)
    return message

# 2.1. TESTS FOR padd_message FUNCTION

In [253]:
# TEST 1
print('Message', '\'F.Limani\'', 'padded looks like:', padd_message('F.Limani'))

# TEST 2
print('Message', '\'Hey\'', 'padded looks like:', padd_message('Hey'))

# TEST 3
print('Message', '\'Fisnik Limani\'', 'padded looks like:', padd_message('Fisnik Limani'))

Message 'F.Limani' padded looks like: F.Limani
Message 'Hey' padded looks like: Hey00000
Message 'Fisnik Limani' padded looks like: Fisnik Limani000


# 3. ECB MODE MODULE

## 3.1. ECB MODE ENCRYPTION

In [335]:
def encryptECB(message, key):
    """
        message - ASCII
        key - HEX
    """
    message = padd_message(message)
    text = _bin_to_hex(_ascii_to_bin(message))
    keys = key_scheduler(key)
    
    blocks = []
    for i in range(0, len(text), 16):
        blocks.append(text[i:i+16])

    e_list = []
    for block in blocks:
        e_list.append(_cipher(_hex_to_bin(block), keys))


    # Combine chunks back into a single string
    out = ""
    for block in e_list:
        out += _bin_to_ascii(block)    # ASCII output
        # out += hex(int(block, 2))      # Hex output
        # out += block                    # Binary output
    return out

## 3.2. ECB MODE DECRYPTION

In [336]:
def decryptECB(cipher, key):
    """
        cipher - ASCII
        key - HEX
    """
    cipher = padd_message(cipher)
    cipher = _ascii_to_bin(cipher)
    keys = key_scheduler(key)
    # print(cipher)
    # Split text up into 64-bit chunks
    blocks = []
    for i in range(0, len(cipher), 64):
        blocks.append(cipher[i:i+64])
    
    d_list = []
    # Decrypt chunks
    for block in blocks:
        d_list.append(_plaintext(block, keys))

    #print(d_list)
    # Combine chunks back into a single string
    out = ""
    for block in d_list:
        out += block
        
    return _bin_to_ascii(out)

## 3.3. ECB MODE TESTS

In [337]:
k = 'FFFFFFFFFFFFFFFF'
m = 'Cryptography is a very interesting !!!!!'
cipher = encryptECB(m,k)
print('CIPHER:', cipher)
message = decryptECB(cipher, k)
print('MESSAGE:', message)

CIPHER: ÒnçØa>PZQHgÏzYÇ*ËF¼ï$6ßif8Llå-
MESSAGE: Cryptography is a very interesting !!!!!


In [338]:
k = '123456789ABCDEF'
m = 'Fisnik loves Cryptography'
cipher = encryptECB(m,k)
print('CIPHER:', cipher)
message = decryptECB(cipher, k)
print('MESSAGE:', message)

CIPHER: í¹û»å$mf#Ì²=¡àu¹%µËñHÝHsÀ
MESSAGE: Fisnik loves Cryptography0000000


# 4. CBC MODE MODULE

## 4.1. CBC MODE ENCRYPTION

In [339]:
def encryptCBC(message, key, iv):
    """
        message - ASCII
        key - HEX
        iv - HEX
    """
    message = padd_message(message)
    text = _bin_to_hex(_ascii_to_bin(message))
    keys = key_scheduler(key)
    iv = _hex_to_bin(iv)
    
    # print(text)
    blocks = []
    for i in range(0, len(text), 16):
        blocks.append(text[i:i+16])
        
    # bin(int(ln, 2) ^ int(f(rn, keys[i]), 2))[2:].zfill(32)
    e_list = []
    prev_output = iv
    for i in range(len(blocks)):
        block = _hex_to_bin(blocks[i])
        block = bin(int(prev_output, 2) ^ int(block, 2))[2:].zfill(64)
        encrypted = _cipher(block, keys)
        e_list.append(encrypted)
        prev_output = encrypted

    # Combine chunks back into a single string
    out = ""
    for block in e_list:
        out += _bin_to_ascii(block)    # ASCII output
        # out += hex(int(block, 2))[2:]      # Hex output
        # out += block                    # Binary output
    return out

## 4.2. CBC MODE DECRYPTION

In [340]:
def decryptCBC(cipher, key, iv):
    """
        message - ASCII
        key - HEX
        iv - HEX
    """
    cipher = padd_message(cipher)
    # cipher = _ascii_to_bin(cipher)
    iv = _hex_to_bin(iv)
    cipher = _ascii_to_bin(cipher)
    # print(len(cipher))
    keys = key_scheduler(key)
    # print(cipher)
    # Split text up into 64-bit chunks
    blocks = []
    for i in range(0, len(cipher), 64):
        blocks.append(cipher[i:i+64])
    
    d_list = []
    # Decrypt chunks
    previous_input = iv
    for i in range(len(blocks)):
        # print(blocks[i])
        decrypted = _plaintext(blocks[i], keys)
        block = bin(int(previous_input, 2) ^ int(decrypted, 2))[2:].zfill(64)
        d_list.append(block)
        previous_input = blocks[i]

    #print(d_list)
    # Combine chunks back into a single string
    out = ""
    for block in d_list:
        out += block
        
    return _bin_to_ascii(out)

## 4.3. CBC MODE TESTS

In [341]:
k = 'FFFFFFFFFFFFFFFF'
iv = '1111111111111111'
m = 'F.Limani'

cipher = encryptCBC(m,k, iv)
print('CIPHER:', cipher)
message = decryptCBC(cipher, k, iv)
print('MESSAGE:', message)

CIPHER: ÈÉe#Åâ
MESSAGE: F.Limani


In [398]:
k = '123456789ABCDEF'
iv = '123456789ABCDEF'
m = 'Fisnik loves Cryptography'

cipher = encryptCBC(m,k, iv)
print('CIPHER:', cipher)
message = decryptCBC(cipher, k, iv)
print('MESSAGE:', message)

)ê@rV
MESSAGE: Fisnik loves Cryptography0000000


# 5. FUNCTION swap_blocks(cipher, b1, b2)

- Function that swaps values of blocks within the cipher corresponding to indexes b1 and b2

In [377]:
def swap_blocks(cipher, b1, b2):
    """
        cipher - ascii
        b1 - index of first block
        b2 - index of second block
    """
    cipher = _ascii_to_bin(cipher)
    blocks = []
    for i in range(0, len(cipher), 64):
        blocks.append(cipher[i:i+64])
    
    #Swap
    block_to_swap = blocks[b1]
    blocks[b1] = blocks[b2]
    blocks[b2] = block_to_swap
    
    out = ""
    for block in blocks:
        out += block
    
    return _bin_to_ascii(out)

# 5.1. TESTS FOR FUNCTION swap_blocks(cipher, b1, b2) 

In [344]:
cipher='AAAAAAAABBBBBBBBCCCCCCCCDDDDDDDDEEEEEEEE'

print('Message:', cipher, '\n')
print('Swap block 0 and 4:', swap_blocks(cipher, 0, 4), '\n')
print('Swap block 1 and 2:', swap_blocks(cipher, 1, 2), '\n')
print('Swap block 3 and 0:', swap_blocks(cipher, 3, 0), '\n')

Message: AAAAAAAABBBBBBBBCCCCCCCCDDDDDDDDEEEEEEEE 

Swap block 0 and 4: EEEEEEEEBBBBBBBBCCCCCCCCDDDDDDDDAAAAAAAA 

Swap block 1 and 2: AAAAAAAACCCCCCCCBBBBBBBBDDDDDDDDEEEEEEEE 

Swap block 3 and 0: DDDDDDDDBBBBBBBBCCCCCCCCAAAAAAAAEEEEEEEE 



# 6. FUNCTION remove_block(cipher, b)

- Function that removes block from the cipher corresponding to index b

In [376]:
def remove_block(cipher, b):
    """
        cipher - ascii
        b - index of block to be removed
    """
    cipher = _ascii_to_bin(cipher)
    blocks = []
    for i in range(0, len(cipher), 64):
        blocks.append(cipher[i:i+64])
    
    #Remove
    new_blocks = []
    for i in range(len(blocks)):
        if i != b:
            new_blocks.append(blocks[i])
    
    out = ""
    for block in new_blocks:
        out += block
        
    return _bin_to_ascii(out)

# 6.1. TESTS FOR FUNCTION remove_block(cipher, b)

In [346]:
cipher='AAAAAAAABBBBBBBBCCCCCCCCDDDDDDDDEEEEEEEE'

print('Message:', cipher, '\n')
print('Remove block 0:', remove_block(cipher, 0), '\n')
print('Remove block 2:', remove_block(cipher, 2), '\n')
print('Remove block 4:', remove_block(cipher, 4), '\n')

Message: AAAAAAAABBBBBBBBCCCCCCCCDDDDDDDDEEEEEEEE 

Remove block 0: BBBBBBBBCCCCCCCCDDDDDDDDEEEEEEEE 

Remove block 2: AAAAAAAABBBBBBBBDDDDDDDDEEEEEEEE 

Remove block 4: AAAAAAAABBBBBBBBCCCCCCCCDDDDDDDD 



# 7. CUT-AND-PASTE ATTACK

#### MESSAGE: ZDRAVO JAS SUM  FISNIK  LIMANI  
#### BLOCKS: ZDRAVO J|AS SUM  |FISNIK  |LIMANI  |

In [371]:
message = 'ZDRAVO JAS SUM  FISNIK  LIMANI  '
key = '123456789ABCDEF'
iv = '123456789ABCDEF'

# 7.1. ATTACK BY SWAPPING BLOCKS ON ECB MODE

In [375]:
ecb_encrypted = encryptECB(message, key)
print('Messsage: ', message)
print('Encrypted:', ecb_encrypted)
swapped_2_3 = swap_blocks(ecb_encrypted, 2, 3)
print('Swapped blocks 2 and 3:', swapped_2_3)
ecb_decrypted = decryptECB(swapped_2_3, key)
print('Decrypt now:', ecb_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: >ö¢'¯UP(1	JÜEÛ»ÅUÇ½ÎÑ\äÌ)¼]
Swapped blocks 2 and 3: >ö¢'¯UP(1	JÜÎÑ\äÌ)¼]EÛ»ÅUÇ½
Decrypt now: ZDRAVO JAS SUM  LIMANI  FISNIK  


## Conclusion
- In ECB mode, if we swap two blocks, then there will be swapped original blocks corresponding to those encrypted blocks too!
- So attack succeded!

# 7.2. ATTACK BY REMOVING BLOCK ON ECB MODE

In [397]:
ecb_encrypted = encryptECB(message, key)
print('Messsage: ', message)
print('Encrypted:', ecb_encrypted)
remove_3 = remove_block(ecb_encrypted, 3)
print('Removed block 3:', remove_3)
ecb_decrypted = decryptECB(remove_3, key)
print('Decrypt now:', ecb_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: >ö¢'¯UP(1	JÜEÛ»ÅUÇ½ÎÑ\äÌ)¼]
Removed block 3: >ö¢'¯UP(1	JÜEÛ»ÅUÇ½
Decrypt now: ZDRAVO JAS SUM  FISNIK  


## Conclusion
- In ECB mode, if we remove a block, then there will be removed the original block corresponding to that encrypted block too!
- So attack succeded!

# 7.3. ATTACK BY SWAPPING BLOCKS ON CBC MODE

In [374]:
cbc_encrypted = encryptCBC(message, key, iv)
print('Messsage: ', message)
print('Encrypted:', cbc_encrypted)
swapped_2_3 = swap_blocks(cbc_encrypted, 2, 3)
print('Swapped blocks 2 and 3:', swapped_2_3)
cbc_decrypted = decryptCBC(swapped_2_3, key, iv)
print('Decrypt now:', cbc_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: è¥6ã¡£Às°"ý¦úT²Yßã/úp~føÑJ
Swapped blocks 2 and 3: è¥6ã¡£Às°"ý¦p~føÑJúT²Yßã/ú
Decrypt now: ZDRAVO JAS SUM  Ýâk!ò|ÍCÌ


## Conclusion
- In CBC mode, if we swap two blocks, then there will be obtained an unclear decryption of that text and all this is due to initial vector that propagates some changes to later blocks using encryption of previous blocks
- So attack didn't succeed (or partially succeded, because swapping was done to the blocks in the end, so only those blocks are affected)!

In [385]:
cbc_encrypted = encryptCBC(message, key, iv)
print('Messsage: ', message)
print('Encrypted:', cbc_encrypted)
swapped_0_1 = swap_blocks(cbc_encrypted, 0, 1)
print('Swapped blocks 0 and 1:', swapped_0_1)
cbc_decrypted = decryptCBC(swapped_0_1, key, iv)
print('Decrypt now:', cbc_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: è¥6ã¡£Às°"ý¦úT²Yßã/úp~føÑJ
Swapped blocks 0 and 1: £Às°"ý¦è¥6ã¡úT²Yßã/úp~føÑJ
Decrypt now: ÒêÇÇqë°J|ÑdaÒñ_>'LIMANI  


In [396]:
cbc_encrypted = encryptCBC(message, key, iv)
print('Messsage: ', message)
print('Encrypted:', cbc_encrypted)
swapped_1_2 = swap_blocks(cbc_encrypted, 1, 2)
print('Swapped blocks 0 and 1:', swapped_1_2)
cbc_decrypted = decryptCBC(swapped_0_1, key, iv)
print('Decrypt now:', cbc_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: è¥6ã¡£Às°"ý¦úT²Yßã/úp~føÑJ
Swapped blocks 0 and 1: è¥6ã¡úT²Yßã/ú£Às°"ý¦p~føÑJ
Decrypt now: ZDRAVO JdaÒñ_>':ï¯ì{Ýâk!ò|


## Conclusion
- In CBC mode, if we swap two blocks in the beginning, then more (but not until the end) of the following blocks will be affected!
- So attack won't (or partially) succeed!

# 7.4. ATTACK BY REMOVING BLOCK ON CBC MODE

In [388]:
cbc_encrypted = encryptCBC(message, key, iv)
print('Messsage: ', message)
print('Encrypted:', cbc_encrypted)
remove_3 = remove_block(cbc_encrypted, 3)
print('Removed block 3:', remove_3)
cbc_decrypted = decryptCBC(remove_3, key, iv)
print('Decrypt now:', cbc_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: è¥6ã¡£Às°"ý¦úT²Yßã/úp~føÑJ
Removed block 3: è¥6ã¡£Às°"ý¦úT²Yßã/ú
Decrypt now: ZDRAVO JAS SUM  FISNIK  


In [393]:
cbc_encrypted = encryptCBC(message, key, iv)
print('Messsage: ', message)
print('Encrypted:', cbc_encrypted)
remove_2 = remove_block(cbc_encrypted, 2)
print('Removed block 2:', remove_2)
cbc_decrypted = decryptCBC(remove_2, key, iv)
print('Decrypt now:', cbc_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: è¥6ã¡£Às°"ý¦úT²Yßã/úp~føÑJ
Removed block 2: è¥6ã¡£Às°"ý¦p~føÑJ
Decrypt now: ZDRAVO JAS SUM  Ýâk!ò|


In [394]:
cbc_encrypted = encryptCBC(message, key, iv)
print('Messsage: ', message)
print('Encrypted:', cbc_encrypted)
remove_1 = remove_block(cbc_encrypted, 1)
print('Removed block 1:', remove_1)
cbc_decrypted = decryptCBC(remove_1, key, iv)
print('Decrypt now:', cbc_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: è¥6ã¡£Às°"ý¦úT²Yßã/úp~føÑJ
Removed block 1: è¥6ã¡úT²Yßã/úp~føÑJ
Decrypt now: ZDRAVO JdaÒñ_>'LIMANI  


In [395]:
cbc_encrypted = encryptCBC(message, key, iv)
print('Messsage: ', message)
print('Encrypted:', cbc_encrypted)
remove_0 = remove_block(cbc_encrypted, 0)
print('Removed block 0:', remove_0)
cbc_decrypted = decryptCBC(remove_0, key, iv)
print('Decrypt now:', cbc_decrypted)

Messsage:  ZDRAVO JAS SUM  FISNIK  LIMANI  
Encrypted: è¥6ã¡£Às°"ý¦úT²Yßã/úp~føÑJ
Removed block 0: £Às°"ý¦úT²Yßã/úp~føÑJ
Decrypt now: ÒêÇÇqFISNIK  LIMANI  


## Conclusion
- In CBC mode, if we remove a block, then only the immediate following block will be affected!
- So attack won't (or partially) succeed!